# Intro
Aim here is to find breakpoint for samples that appear to be duplicated around plasmepsin I-III, but not with the same breakpoints as most other samples. See email from Rob 21/09/2016 17:15.

I installed IGV on maslrv and ran with:

ssh -C -X -c blowfish malsrv2

/nfs/team112_internal/rp7/opt/igv/IGV_2.3.81/igv.sh

Load genome (Genomes/Load Genome from File) with /lustre/scratch116/malaria/pfalciparum/resources/Pfalciparum.genome.fasta

Load bam using manifest below to find correct file

Set "Show soft-clipped bases" on Alignments tab (View/Preferences)

Zoom to Pf3D7_14_v3:280,000-310,000

Later decided that running IGV over -X was too slow so copied bam files to MacBook


# Plan
- Try searching for "1-sided" reads near long homopolymer-A
- Also search for reads spanning homopolymer, but with variable length polyA, and 1-5 bases downstream
- pysam code to find number and proportion of pair where one end in one region, with particular orientation, and mate in second region with expected orientation
- generalise the above code to find all peaks of faceaway reads in larger region
- HMM code to search for coverage changes in whole region
- combine evidence from all above approaches


- pysam code to pull out all soft-clipped reads in a region?


In [1]:
%run _standard_imports.ipynb

python 3.4.5 |Anaconda 2.2.0 (64-bit)| (default, Jul  2 2016, 17:47:47) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
numpy 1.11.1
scipy 0.18.0
pandas 0.15.2
numexpr 2.3.1
pysam 0.8.3
pysamstats 0.23
petl 1.1.0
petlx 1.0.3
vcf 0.6.7
vcfnp 2.3.0.dev0
h5py 2.4.0
tables 3.1.1


In [5]:
from Bio.Seq import Seq


In [89]:
diff_breakpoint_samples = ['PH0243-C', 'PH0247-C', 'PH0484-C', 'PH0906-C', 'PH0912-C']
mdr1_samples = ['PH0254-C']

In [36]:
# Note we used same samples as were previously used for HRP deletions. See 20160621_HRP_sample_metadata.ipynb
manifest_5_0_fn = '/nfs/team112_internal/rp7/data/Pf/hrp/metadata/hrp_manifest_20160621.txt'
# Note the following file created whilst 6.0 build still in progress, so don't have final sample bams
manifest_6_0_fn = '/nfs/team112_internal/rp7/data/Pf/6_0/metadata/plasmepsin_manifest_6_0_20160922.txt'
scratch_dir = '/lustre/scratch109/malaria/rp7/data/ppq'

genome_fn = '/lustre/scratch116/malaria/pfalciparum/resources/Pfalciparum.genome.fasta'

In [38]:
genome = SeqIO.to_dict(SeqIO.parse(genome_fn, "fasta"))
# , key_function=get_accession)

In [40]:
genome.keys()

dict_keys(['Pf3D7_03_v3', 'Pf3D7_14_v3', 'Pf3D7_02_v3', 'Pf3D7_05_v3', 'Pf3D7_11_v3', 'Pf3D7_01_v3', 'Pf3D7_04_v3', 'Pf3D7_07_v3', 'Pf_M76611', 'Pf3D7_10_v3', 'Pf3D7_13_v3', 'Pf3D7_06_v3', 'Pf3D7_API_v3', 'Pf3D7_12_v3', 'Pf3D7_09_v3', 'Pf3D7_08_v3'])

In [41]:
genome['Pf3D7_14_v3'].count()

SeqRecord(seq=Seq('tgaaccctaaaccctaaaccctaaaccctaaacccctaaaccctaaaccctgaa...gtt', SingleLetterAlphabet()), id='Pf3D7_14_v3', name='Pf3D7_14_v3', description='Pf3D7_14_v3', dbxrefs=[])

In [3]:
tbl_manifest = etl.fromtsv(manifest_fn)
print(len(tbl_manifest.data()))
tbl_manifest

7979


path,sample,oxford_src_code,oxford_donor_code,dataset,type,region_code,country_code,country_name,sub_continent,sub_continent_name,sub_continent_number,bases_of_5X_coverage,mean_coverage,bases_of_1X_coverage,bases_of_10X_coverage,bases_of_20X_coverage
/lustre/scratch109/malaria/pfalciparum/output/a/1/d/0/265637/1_bam_merge/pe.1.bam,QT0001-C,CM001-J0,CM001,pf_community_5_1,dna,BJ,BJ,Benin,WAF,West Africa,1,22429396,110.27,22866093,21829039,20453074
/lustre/scratch109/malaria/pfalciparum/output/f/1/2/c/265638/1_bam_merge/pe.1.bam,QT0001-CW4,CM001-H8,CM001,pf_community_5_1,MDA,BJ,BJ,Benin,WAF,West Africa,1,21661068,96.55,22734327,20463738,18329425
/lustre/scratch109/malaria/pfalciparum/output/0/2/0/6/265639/1_bam_merge/pe.1.bam,QT0001-CW5,CM001-H16,CM001,pf_community_5_1,MDA,BJ,BJ,Benin,WAF,West Africa,1,21409846,68.65,22651241,19897257,17210983
/lustre/scratch109/malaria/pfalciparum/output/d/a/a/f/265640/1_bam_merge/pe.1.bam,QT0001-CW6,CM001-H24,CM001,pf_community_5_1,MDA,BJ,BJ,Benin,WAF,West Africa,1,6065338,4.34,16902776,1391631,106667
/lustre/scratch109/malaria/pfalciparum/output/6/7/b/4/265641/1_bam_merge/pe.1.bam,QT0002-C,CM002-J0,CM002,pf_community_5_1,dna,BJ,BJ,Benin,WAF,West Africa,1,22434258,92.75,22854818,21789730,20157016


In [91]:
tbl_bams_5_0 = tbl_manifest.selectin('sample', diff_breakpoint_samples)
tbl_bams_5_0.display()

path,sample,oxford_src_code,oxford_donor_code,dataset,type,region_code,country_code,country_name,sub_continent,sub_continent_name,sub_continent_number,bases_of_5X_coverage,mean_coverage,bases_of_1X_coverage,bases_of_10X_coverage,bases_of_20X_coverage
/lustre/scratch109/malaria/pfalciparum/output/8/c/f/3/61719/1_bam_merge/pe.1.bam,PH0243-C,PF823,PF823,pf_community_5_0,dna,WKH,KH,Cambodia,ESEA,Eastern south-east Asia,5,22691239,78.62,22939256,22477810,21999583
/lustre/scratch109/malaria/pfalciparum/output/3/4/d/d/44281/1_bam_merge/pe.1.bam,PH0247-C,PF832,PF832,pf_community_5_0,dna,WKH,KH,Cambodia,ESEA,Eastern south-east Asia,5,22600719,58.98,22917058,22288524,21483373
/lustre/scratch109/malaria/pfalciparum/output/5/3/b/4/44503/1_bam_merge/pe.1.bam,PH0484-C,668-2,668-2,pf_community_5_0,dna,WKH,KH,Cambodia,ESEA,Eastern south-east Asia,5,22695688,77.37,22961714,22466908,21890185
/lustre/scratch109/malaria/pfalciparum/output/4/5/7/8/44843/1_bam_merge/pe.1.bam,PH0906-C,KH004-068,KH004-068,pf_community_5_0,dna,WKH,KH,Cambodia,ESEA,Eastern south-east Asia,5,21907288,22.97,22843706,19699277,13525111
/lustre/scratch109/malaria/pfalciparum/output/9/7/b/2/44848/1_bam_merge/pe.1.bam,PH0912-C,KH004-076,KH004-076,pf_community_5_0,dna,WKH,KH,Cambodia,ESEA,Eastern south-east Asia,5,22206091,31.95,22778189,21046597,17180194


In [92]:
tbl_manifest.selectin('sample', mdr1_samples)


path,sample,oxford_src_code,oxford_donor_code,dataset,type,region_code,country_code,country_name,sub_continent,sub_continent_name,sub_continent_number,bases_of_5X_coverage,mean_coverage,bases_of_1X_coverage,bases_of_10X_coverage,bases_of_20X_coverage
/lustre/scratch109/malaria/pfalciparum/output/f/c/3/e/44288/1_bam_merge/pe.1.bam,PH0254-C,PF949_LP,PF949_LP,pf_community_5_0,dna,WKH,KH,Cambodia,ESEA,Eastern south-east Asia,5,22680518,82.68,22955284,22461797,21957452


# Results
PH0243-C: polyA (36bp) at 283,034 to polyA (30bp) at 300,493 (17kb)
Think PH0247-C and PH0484-C have the same breakpoints

PH0906-C: polyA (36bp) at 283,034 to polyA (30bp) at 362,990 (80kb)
Think PH0912-C has same breakpoints


In [43]:
genome['Pf3D7_14_v3'][283032:283070].seq

Seq('caaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaac', SingleLetterAlphabet())

In [48]:
genome['Pf3D7_14_v3'].seq.count(genome['Pf3D7_14_v3'][283020:283069].seq)

1

In [60]:
temp = list(genome.keys())
temp.sort()
temp


['Pf3D7_01_v3',
 'Pf3D7_02_v3',
 'Pf3D7_03_v3',
 'Pf3D7_04_v3',
 'Pf3D7_05_v3',
 'Pf3D7_06_v3',
 'Pf3D7_07_v3',
 'Pf3D7_08_v3',
 'Pf3D7_09_v3',
 'Pf3D7_10_v3',
 'Pf3D7_11_v3',
 'Pf3D7_12_v3',
 'Pf3D7_13_v3',
 'Pf3D7_14_v3',
 'Pf3D7_API_v3',
 'Pf_M76611']

In [61]:
sorted(genome.keys())

['Pf3D7_01_v3',
 'Pf3D7_02_v3',
 'Pf3D7_03_v3',
 'Pf3D7_04_v3',
 'Pf3D7_05_v3',
 'Pf3D7_06_v3',
 'Pf3D7_07_v3',
 'Pf3D7_08_v3',
 'Pf3D7_09_v3',
 'Pf3D7_10_v3',
 'Pf3D7_11_v3',
 'Pf3D7_12_v3',
 'Pf3D7_13_v3',
 'Pf3D7_14_v3',
 'Pf3D7_API_v3',
 'Pf_M76611']

In [81]:
import re
for chrom in sorted(genome.keys()):
#     print(chrom, genome[chrom].seq.count(genome['Pf3D7_14_v3'][283027:283069].seq))
#     print(chrom, genome[chrom].seq.find('agaagtaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa'))
    print(chrom, [m.start() for m in re.finditer('agaagtaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa', str(genome[chrom].seq))])

print()

for chrom in sorted(genome.keys()):
    print(chrom, [m.start() for m in re.finditer('aaaaaaagaagtaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa', str(genome[chrom].seq))])

print()

for chrom in sorted(genome.keys()):
    print(chrom, [m.start() for m in re.finditer('aaaaagaagtaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa', str(genome[chrom].seq))])

    

Pf3D7_01_v3 []
Pf3D7_02_v3 []
Pf3D7_03_v3 []
Pf3D7_04_v3 []
Pf3D7_05_v3 []
Pf3D7_06_v3 []
Pf3D7_07_v3 []
Pf3D7_08_v3 []
Pf3D7_09_v3 []
Pf3D7_10_v3 []
Pf3D7_11_v3 []
Pf3D7_12_v3 []
Pf3D7_13_v3 []
Pf3D7_14_v3 [128248, 362983]
Pf3D7_API_v3 []
Pf_M76611 []

Pf3D7_01_v3 []
Pf3D7_02_v3 []
Pf3D7_03_v3 []
Pf3D7_04_v3 []
Pf3D7_05_v3 []
Pf3D7_06_v3 []
Pf3D7_07_v3 []
Pf3D7_08_v3 []
Pf3D7_09_v3 []
Pf3D7_10_v3 []
Pf3D7_11_v3 []
Pf3D7_12_v3 []
Pf3D7_13_v3 []
Pf3D7_14_v3 [362977]
Pf3D7_API_v3 []
Pf_M76611 []

Pf3D7_01_v3 []
Pf3D7_02_v3 []
Pf3D7_03_v3 []
Pf3D7_04_v3 []
Pf3D7_05_v3 []
Pf3D7_06_v3 []
Pf3D7_07_v3 []
Pf3D7_08_v3 []
Pf3D7_09_v3 []
Pf3D7_10_v3 []
Pf3D7_11_v3 []
Pf3D7_12_v3 []
Pf3D7_13_v3 []
Pf3D7_14_v3 [362979]
Pf3D7_API_v3 []
Pf_M76611 []


In [84]:
for chrom in sorted(genome.keys()):
    print(chrom, [m.start() for m in re.finditer('aaatgaagggaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa', str(genome[chrom].seq))])


Pf3D7_01_v3 []
Pf3D7_02_v3 []
Pf3D7_03_v3 []
Pf3D7_04_v3 []
Pf3D7_05_v3 []
Pf3D7_06_v3 []
Pf3D7_07_v3 []
Pf3D7_08_v3 []
Pf3D7_09_v3 []
Pf3D7_10_v3 []
Pf3D7_11_v3 []
Pf3D7_12_v3 []
Pf3D7_13_v3 []
Pf3D7_14_v3 [300482]
Pf3D7_API_v3 []
Pf_M76611 []


In [85]:
breakpoint_sequences = collections.OrderedDict()
breakpoint_sequences['0bp'] = 'GATAATCACAC'
breakpoint_sequences['1bp'] = 'CGATAATCACACT'
breakpoint_sequences['5bp'] = 'ATTACGATAATCACACTGTTG'
breakpoint_sequences['10bp'] = 'TTATGATTACGATAATCACACTGTTGGTTTC'
breakpoint_sequences['15bp'] = 'ACCGTTTATGATTACGATAATCACACTGTTGGTTTCGCCCT'
breakpoint_sequences['20bp'] = 'ATTTTACCGTTTATGATTACGATAATCACACTGTTGGTTTCGCCCTTGCCA'


In [86]:
breakpoint_sequences['0bp'].lower()

'gataatcacac'

In [87]:
for breakpoint_sequence in [x.lower() for x in breakpoint_sequences.values()]:
    print(breakpoint_sequence)
    for chrom in sorted(genome.keys()):
        print(chrom, [m.start() for m in re.finditer(breakpoint_sequence, str(genome[chrom].seq))])


gataatcacac
Pf3D7_01_v3 []
Pf3D7_02_v3 []
Pf3D7_03_v3 []
Pf3D7_04_v3 []
Pf3D7_05_v3 []
Pf3D7_06_v3 []
Pf3D7_07_v3 []
Pf3D7_08_v3 []
Pf3D7_09_v3 []
Pf3D7_10_v3 []
Pf3D7_11_v3 []
Pf3D7_12_v3 [976438, 1788850]
Pf3D7_13_v3 []
Pf3D7_14_v3 [289610, 298781]
Pf3D7_API_v3 []
Pf_M76611 []
cgataatcacact
Pf3D7_01_v3 []
Pf3D7_02_v3 []
Pf3D7_03_v3 []
Pf3D7_04_v3 []
Pf3D7_05_v3 []
Pf3D7_06_v3 []
Pf3D7_07_v3 []
Pf3D7_08_v3 []
Pf3D7_09_v3 []
Pf3D7_10_v3 []
Pf3D7_11_v3 []
Pf3D7_12_v3 []
Pf3D7_13_v3 []
Pf3D7_14_v3 []
Pf3D7_API_v3 []
Pf_M76611 []
attacgataatcacactgttg
Pf3D7_01_v3 []
Pf3D7_02_v3 []
Pf3D7_03_v3 []
Pf3D7_04_v3 []
Pf3D7_05_v3 []
Pf3D7_06_v3 []
Pf3D7_07_v3 []
Pf3D7_08_v3 []
Pf3D7_09_v3 []
Pf3D7_10_v3 []
Pf3D7_11_v3 []
Pf3D7_12_v3 []
Pf3D7_13_v3 []
Pf3D7_14_v3 []
Pf3D7_API_v3 []
Pf_M76611 []
ttatgattacgataatcacactgttggtttc
Pf3D7_01_v3 []
Pf3D7_02_v3 []
Pf3D7_03_v3 []
Pf3D7_04_v3 []
Pf3D7_05_v3 []
Pf3D7_06_v3 []
Pf3D7_07_v3 []
Pf3D7_08_v3 []
Pf3D7_09_v3 []
Pf3D7_10_v3 []
Pf3D7_11_v3 []
Pf3D7_12_

In [88]:
for breakpoint_sequence in ['cgataatcacact', 'cgataatcacac', 'gataatcacact', 'gataatcacac']:
    print(breakpoint_sequence)
    for chrom in sorted(genome.keys()):
        print(chrom, [m.start() for m in re.finditer(breakpoint_sequence, str(genome[chrom].seq))])


cgataatcacact
Pf3D7_01_v3 []
Pf3D7_02_v3 []
Pf3D7_03_v3 []
Pf3D7_04_v3 []
Pf3D7_05_v3 []
Pf3D7_06_v3 []
Pf3D7_07_v3 []
Pf3D7_08_v3 []
Pf3D7_09_v3 []
Pf3D7_10_v3 []
Pf3D7_11_v3 []
Pf3D7_12_v3 []
Pf3D7_13_v3 []
Pf3D7_14_v3 []
Pf3D7_API_v3 []
Pf_M76611 []
cgataatcacac
Pf3D7_01_v3 []
Pf3D7_02_v3 []
Pf3D7_03_v3 []
Pf3D7_04_v3 []
Pf3D7_05_v3 []
Pf3D7_06_v3 []
Pf3D7_07_v3 []
Pf3D7_08_v3 []
Pf3D7_09_v3 []
Pf3D7_10_v3 []
Pf3D7_11_v3 []
Pf3D7_12_v3 []
Pf3D7_13_v3 []
Pf3D7_14_v3 [298780]
Pf3D7_API_v3 []
Pf_M76611 []
gataatcacact
Pf3D7_01_v3 []
Pf3D7_02_v3 []
Pf3D7_03_v3 []
Pf3D7_04_v3 []
Pf3D7_05_v3 []
Pf3D7_06_v3 []
Pf3D7_07_v3 []
Pf3D7_08_v3 []
Pf3D7_09_v3 []
Pf3D7_10_v3 []
Pf3D7_11_v3 []
Pf3D7_12_v3 []
Pf3D7_13_v3 []
Pf3D7_14_v3 [289610]
Pf3D7_API_v3 []
Pf_M76611 []
gataatcacac
Pf3D7_01_v3 []
Pf3D7_02_v3 []
Pf3D7_03_v3 []
Pf3D7_04_v3 []
Pf3D7_05_v3 []
Pf3D7_06_v3 []
Pf3D7_07_v3 []
Pf3D7_08_v3 []
Pf3D7_09_v3 []
Pf3D7_10_v3 []
Pf3D7_11_v3 []
Pf3D7_12_v3 [976438, 1788850]
Pf3D7_13_v3 []
Pf3D7_14_

In [68]:
genome['Pf3D7_14_v3'][283027:283069].seq

Seq('ccacacaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa', SingleLetterAlphabet())

In [100]:
for breakpoint_sequence in ['aaaaaaaaaaaaaaaaaaaa']:
    print(breakpoint_sequence)
    for chrom in ['Pf3D7_05_v3']:
#     for chrom in sorted(genome.keys()):
        print(chrom, [m.start() for m in re.finditer(breakpoint_sequence, str(genome[chrom].seq))])


aaaaaaaaaaaaaaaaaaaa
Pf3D7_05_v3 [18878, 18961, 19044, 26493, 42610, 49788, 49910, 53181, 54597, 56896, 57712, 59501, 61174, 65906, 67218, 70166, 80301, 83465, 83735, 85498, 90088, 90989, 95083, 97161, 97754, 108443, 108654, 109271, 109769, 112663, 116196, 121726, 124265, 126033, 129710, 133839, 134112, 136256, 137528, 138878, 140043, 141561, 141955, 141975, 142839, 146325, 151292, 151630, 165678, 166839, 167571, 174645, 174665, 177091, 180277, 182166, 187502, 187740, 205006, 218763, 219085, 220651, 225919, 233088, 233678, 239088, 243962, 244850, 245730, 250895, 254033, 254511, 257804, 263301, 263321, 279752, 280251, 283386, 283635, 284405, 291186, 291533, 293534, 294367, 294453, 296003, 296053, 296571, 297740, 297987, 300991, 301401, 305234, 306659, 307216, 310198, 310349, 316114, 316585, 319453, 319552, 321919, 323844, 326820, 327413, 334950, 345104, 362740, 363491, 365747, 377970, 379297, 390317, 405750, 408487, 409193, 415449, 418956, 421432, 431413, 431630, 439871, 441489, 442148,

# Copy bams to MacBook
Decided to copy both Pf 5.0 bams (mapped with bwa aln) and 6.0 bams (mapped with bwa mem)


In [19]:
print("mkdir -p /lustre/scratch109/malaria/rp7/data/ppq/bams/5_0/")
for rec in tbl_bams_5_0.data():
    original_bam = rec[0]
    macbook_bam = "%s/bams/5_0/%s.bam" % (scratch_dir, rec[1])
    print("scp malsrv2:%s %s" % (original_bam.replace('.bam', '.bam.bai'), macbook_bam.replace('.bam', '.bam.bai')))
    print("scp malsrv2:%s %s" % (original_bam, macbook_bam))


mkdir -p /lustre/scratch109/malaria/rp7/data/ppq/bams/5_0/
scp malsrv2:/lustre/scratch109/malaria/pfalciparum/output/8/c/f/3/61719/1_bam_merge/pe.1.bam.bai /lustre/scratch109/malaria/rp7/data/ppq/bams/5_0/PH0243-C.bam.bai
scp malsrv2:/lustre/scratch109/malaria/pfalciparum/output/8/c/f/3/61719/1_bam_merge/pe.1.bam /lustre/scratch109/malaria/rp7/data/ppq/bams/5_0/PH0243-C.bam
scp malsrv2:/lustre/scratch109/malaria/pfalciparum/output/3/4/d/d/44281/1_bam_merge/pe.1.bam.bai /lustre/scratch109/malaria/rp7/data/ppq/bams/5_0/PH0247-C.bam.bai
scp malsrv2:/lustre/scratch109/malaria/pfalciparum/output/3/4/d/d/44281/1_bam_merge/pe.1.bam /lustre/scratch109/malaria/rp7/data/ppq/bams/5_0/PH0247-C.bam
scp malsrv2:/lustre/scratch109/malaria/pfalciparum/output/5/3/b/4/44503/1_bam_merge/pe.1.bam.bai /lustre/scratch109/malaria/rp7/data/ppq/bams/5_0/PH0484-C.bam.bai
scp malsrv2:/lustre/scratch109/malaria/pfalciparum/output/5/3/b/4/44503/1_bam_merge/pe.1.bam /lustre/scratch109/malaria/rp7/data/ppq/bams/5_0/

In [23]:
!mkdir -p {os.path.dirname(manifest_6_0_fn)}
!vrpipe-fileinfo --setup pf_60_bqsr --metadata sample > {manifest_6_0_fn}


In [33]:
tbl_manifest_6_0 = etl.fromtsv(manifest_6_0_fn).select(lambda rec: rec[0][-3:] == 'bam')
print(len(tbl_manifest_6_0.data()))
tbl_manifest_6_0

8959


path,sample
/lustre/scratch116/malaria/pfalciparum/output/b/f/a/2/35717/2_gatk_print_reads_gatk3_v2/18140_5_12.pe.markdup.recal.bam,PA0704-CW
/lustre/scratch116/malaria/pfalciparum/output/8/9/2/7/28200/2_gatk_print_reads_gatk3_v2/17726_2_4.pe.markdup.recal.bam,PH1655-C
/lustre/scratch116/malaria/pfalciparum/output/e/1/7/f/34673/2_gatk_print_reads_gatk3_v2/14200_1_11.pe.markdup.recal.bam,PC0318-C
/lustre/scratch116/malaria/pfalciparum/output/3/2/3/4/29163/2_gatk_print_reads_gatk3_v2/6288_8_nonhuman_3.pe.markdup.recal.bam,PF0363-C
/lustre/scratch116/malaria/pfalciparum/output/3/d/e/8/36000/2_gatk_print_reads_gatk3_v2/19564_3_8.pe.markdup.recal.bam,QG0410-C


In [30]:
tbl_bams_6_0 = tbl_manifest_6_0.selectin('sample', diff_breakpoint_samples).sort('sample')
tbl_bams_6_0.display()

path,sample
/lustre/scratch116/malaria/pfalciparum/output/9/5/9/5/27793/2_gatk_print_reads_gatk3_v2/6297_1_nonhuman_7.pe.markdup.recal.bam,PH0243-C
/lustre/scratch116/malaria/pfalciparum/output/b/f/0/3/27727/2_gatk_print_reads_gatk3_v2/6222_8_nonhuman_4.pe.markdup.recal.bam,PH0247-C
/lustre/scratch116/malaria/pfalciparum/output/4/7/6/f/27741/2_gatk_print_reads_gatk3_v2/6263_3_nonhuman_2.pe.markdup.recal.bam,PH0484-C
/lustre/scratch116/malaria/pfalciparum/output/7/2/c/1/30749/2_gatk_print_reads_gatk3_v2/8668_7_16.pe.markdup.recal.bam,PH0906-C
/lustre/scratch116/malaria/pfalciparum/output/0/7/b/0/30744/2_gatk_print_reads_gatk3_v2/8668_7_19.pe.markdup.recal.bam,PH0912-C


In [34]:
print("mkdir -p /lustre/scratch109/malaria/rp7/data/ppq/bams/6_0/")
for rec in tbl_bams_6_0.data():
    original_bam = rec[0]
    macbook_bam = "%s/bams/6_0/%s.bam" % (scratch_dir, rec[1])
    print("scp malsrv2:%s %s" % (original_bam.replace('.bam', '.bai'), macbook_bam.replace('.bam', '.bai')))
    print("scp malsrv2:%s %s" % (original_bam, macbook_bam))


mkdir -p /lustre/scratch109/malaria/rp7/data/ppq/bams/6_0/
scp malsrv2:/lustre/scratch116/malaria/pfalciparum/output/9/5/9/5/27793/2_gatk_print_reads_gatk3_v2/6297_1_nonhuman_7.pe.markdup.recal.bai /lustre/scratch109/malaria/rp7/data/ppq/bams/6_0/PH0243-C.bai
scp malsrv2:/lustre/scratch116/malaria/pfalciparum/output/9/5/9/5/27793/2_gatk_print_reads_gatk3_v2/6297_1_nonhuman_7.pe.markdup.recal.bam /lustre/scratch109/malaria/rp7/data/ppq/bams/6_0/PH0243-C.bam
scp malsrv2:/lustre/scratch116/malaria/pfalciparum/output/b/f/0/3/27727/2_gatk_print_reads_gatk3_v2/6222_8_nonhuman_4.pe.markdup.recal.bai /lustre/scratch109/malaria/rp7/data/ppq/bams/6_0/PH0247-C.bai
scp malsrv2:/lustre/scratch116/malaria/pfalciparum/output/b/f/0/3/27727/2_gatk_print_reads_gatk3_v2/6222_8_nonhuman_4.pe.markdup.recal.bam /lustre/scratch109/malaria/rp7/data/ppq/bams/6_0/PH0247-C.bam
scp malsrv2:/lustre/scratch116/malaria/pfalciparum/output/4/7/6/f/27741/2_gatk_print_reads_gatk3_v2/6263_3_nonhuman_2.pe.markdup.recal.ba

In [93]:
tbl_manifest_6_0.selectin('sample', mdr1_samples)

path,sample
/lustre/scratch116/malaria/pfalciparum/output/0/7/a/3/27761/2_gatk_print_reads_gatk3_v2/6263_6_nonhuman_1.pe.markdup.recal.bam,PH0254-C


In [10]:
import stat
bsub = sh.Command('bsub')

for breakpoint_sequence_name in breakpoint_sequences:
# for breakpoint_sequence_name in ['10bp']:
    breakpoint_reads_dir = '%s/plasmepsin_1_3_7979_samples/%s' % (output_dir, breakpoint_sequence_name)
    !mkdir -p {breakpoint_reads_dir}/results
    !mkdir -p {breakpoint_reads_dir}/scripts
    !mkdir -p {breakpoint_reads_dir}/logs

    breakpoint_sequence = "'%s|%s'" % (
        breakpoint_sequences[breakpoint_sequence_name],
        Seq(breakpoint_sequences[breakpoint_sequence_name]).reverse_complement()
    )
    for rec in tbl_manifest:
        bam_fn = rec[0]
        ox_code = rec[1]
        print('.', end='')
        num_breakpoint_reads_fn = "%s/results/num_breakpoint_reads_%s.txt" % (breakpoint_reads_dir, ox_code)
        if not os.path.exists(num_breakpoint_reads_fn):
            script_fn = "%s/scripts/nbpr_%s.sh" % (breakpoint_reads_dir, ox_code)
            fo = open(script_fn, 'w')
            print('samtools view %s | grep -E %s | wc -l > %s' % (bam_fn, breakpoint_sequence, num_breakpoint_reads_fn), file = fo)
            fo.close()
            st = os.stat(script_fn)
            os.chmod(script_fn, st.st_mode | stat.S_IEXEC)
            bsub(
                '-G', 'malaria-dk',
                '-P', 'malaria-dk',
                '-q', 'normal',
                '-o', '%s/logs/nbpr_%s.out' % (breakpoint_reads_dir, ox_code),
                '-e', '%s/logs/nbpr_%s.err' % (breakpoint_reads_dir, ox_code),
                '-J', 'nbpr_%s' % (ox_code),
                '-R', "'select[mem>4000] rusage[mem=4000]'",
                '-M', '4000',
                script_fn
            )



........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [13]:
tbl_manifest

path,sample,oxford_src_code,oxford_donor_code,dataset,type,region_code,country_code,country_name,sub_continent,sub_continent_name,sub_continent_number,bases_of_5X_coverage,mean_coverage,bases_of_1X_coverage,bases_of_10X_coverage,bases_of_20X_coverage
/lustre/scratch109/malaria/pfalciparum/output/a/1/d/0/265637/1_bam_merge/pe.1.bam,QT0001-C,CM001-J0,CM001,pf_community_5_1,dna,BJ,BJ,Benin,WAF,West Africa,1,22429396,110.27,22866093,21829039,20453074
/lustre/scratch109/malaria/pfalciparum/output/f/1/2/c/265638/1_bam_merge/pe.1.bam,QT0001-CW4,CM001-H8,CM001,pf_community_5_1,MDA,BJ,BJ,Benin,WAF,West Africa,1,21661068,96.55,22734327,20463738,18329425
/lustre/scratch109/malaria/pfalciparum/output/0/2/0/6/265639/1_bam_merge/pe.1.bam,QT0001-CW5,CM001-H16,CM001,pf_community_5_1,MDA,BJ,BJ,Benin,WAF,West Africa,1,21409846,68.65,22651241,19897257,17210983
/lustre/scratch109/malaria/pfalciparum/output/d/a/a/f/265640/1_bam_merge/pe.1.bam,QT0001-CW6,CM001-H24,CM001,pf_community_5_1,MDA,BJ,BJ,Benin,WAF,West Africa,1,6065338,4.34,16902776,1391631,106667
/lustre/scratch109/malaria/pfalciparum/output/6/7/b/4/265641/1_bam_merge/pe.1.bam,QT0002-C,CM002-J0,CM002,pf_community_5_1,dna,BJ,BJ,Benin,WAF,West Africa,1,22434258,92.75,22854818,21789730,20157016


In [5]:
pf_5_0_breakpoint_reads = collections.OrderedDict()

# for breakpoint_sequence_name in breakpoint_sequences:
for breakpoint_sequence_name in ['10bp']:
    pf_5_0_breakpoint_reads[breakpoint_sequence_name] = collections.OrderedDict()
    breakpoint_reads_dir = '%s/plasmepsin_1_3_pf_5_0/%s' % (output_dir, breakpoint_sequence_name)
    for rec in tbl_5_0_manifest:
        bam_fn = rec[0]
        ox_code = rec[2]
        print('.', end='')
        num_breakpoint_reads_fn = "%s/results/num_breakpoint_reads_%s.txt" % (breakpoint_reads_dir, ox_code)
        fi = open(num_breakpoint_reads_fn, 'r')
        pf_5_0_breakpoint_reads[breakpoint_sequence_name][ox_code] = int(fi.read())



........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [15]:
breakpoint_reads_dict = collections.OrderedDict()

for breakpoint_sequence_name in breakpoint_sequences:
    print(breakpoint_sequence_name)
# for breakpoint_sequence_name in ['10bp']:
    breakpoint_reads_dict[breakpoint_sequence_name] = collections.OrderedDict()
    breakpoint_reads_dir = '%s/plasmepsin_1_3_7979_samples/%s' % (output_dir, breakpoint_sequence_name)
    for rec in tbl_manifest.data():
        bam_fn = rec[0]
        ox_code = rec[1]
        print('.', end='')
        num_breakpoint_reads_fn = "%s/results/num_breakpoint_reads_%s.txt" % (breakpoint_reads_dir, ox_code)
        fi = open(num_breakpoint_reads_fn, 'r')
        breakpoint_reads_dict[breakpoint_sequence_name][ox_code] = int(fi.read())



0bp
....................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [17]:
tbl_breakpoint_reads = (etl.wrap(zip(breakpoint_reads_dict['0bp'].keys(), breakpoint_reads_dict['0bp'].values())).pushheader(['ox_code', 'bp_reads_0bp'])
 .join(
        etl.wrap(zip(breakpoint_reads_dict['1bp'].keys(), breakpoint_reads_dict['1bp'].values())).pushheader(['ox_code', 'bp_reads_1bp']),
        key='ox_code')
 .join(
        etl.wrap(zip(breakpoint_reads_dict['5bp'].keys(), breakpoint_reads_dict['5bp'].values())).pushheader(['ox_code', 'bp_reads_5bp']),
        key='ox_code')
 .join(
        etl.wrap(zip(breakpoint_reads_dict['10bp'].keys(), breakpoint_reads_dict['10bp'].values())).pushheader(['ox_code', 'bp_reads_10bp']),
        key='ox_code')
 .join(
        etl.wrap(zip(breakpoint_reads_dict['15bp'].keys(), breakpoint_reads_dict['15bp'].values())).pushheader(['ox_code', 'bp_reads_15bp']),
        key='ox_code')
 .join(
        etl.wrap(zip(breakpoint_reads_dict['20bp'].keys(), breakpoint_reads_dict['20bp'].values())).pushheader(['ox_code', 'bp_reads_20bp']),
        key='ox_code')
)
# tbl_breakpoint_reads.displayall()

In [21]:
tbl_breakpoint_reads.selectgt('bp_reads_10bp', 0).displayall()

ox_code,bp_reads_0bp,bp_reads_1bp,bp_reads_5bp,bp_reads_10bp,bp_reads_15bp,bp_reads_20bp
PD0033-C,1691,12,1,1,1,1
PD0667-C,479,71,63,50,39,34
PD0727-C,349,57,54,49,42,35
PD0729-C,195,23,19,17,14,13
PD1034-C,157,4,4,4,3,1
PD1407-C,299,38,33,32,30,25
PF0117-C,948,3,1,1,1,1
PF0364-C,44,2,2,2,1,0
PH0041-C,1843,327,293,237,176,87
PH0044-C,1368,210,184,156,112,86


In [22]:
tbl_breakpoint_reads.totsv("%s/plasmepsin_1_3_7979_samples.tsv" % output_dir)

In [23]:
tbl_breakpoint_reads.toxlsx("%s/plasmepsin_1_3_7979_samples.xlsx" % output_dir)

In [101]:
2+2

4